In [40]:
from PIL import Image
import os
import random

from PIL import ImagePalette
import numpy as np

In [41]:
res_h = 3
res_w = 4

In [42]:
pitman_list = """
007
009
024
028
034
038
047
049
052
056
060
064
065
080
084
085
087
089
090
093
100
105
111
113
116
121
124
129
134
151
152
158
164
168
179
186
189
190
198
201
207
236
250
256
"""

In [43]:
pitman_list = pitman_list.split('\n')[1:]

In [44]:
list_of_files = []

for folder in os.listdir('256_ObjectCategories/'):
    PATH = '256_ObjectCategories/'+folder
    if folder[:3] in pitman_list:
        os.mkdir('chopped_images/'+folder)
        if os.path.isdir(PATH):
            list_of_files+= [[PATH + '/' + file, folder] for file in 
                             os.listdir('256_ObjectCategories/'+folder)[:50] if 
                             file[-3:] in ['png','jpg','jpeg']]
                
print('Recovered {} images'.format(len(list_of_files)))

Recovered 2200 images


In [45]:
from tqdm import tqdm

count = 0
h = 0
v = 0
usable_files = []
for [file, tag] in tqdm(list_of_files):
    im = Image.open(file)
    
    im_array = np.array(im)
    h = im_array.shape[0]
    v = im_array.shape[1]
    
    ratio = v/h
    
    if h > 600:
        if ratio >= (res_w/res_h - 0.2) and ratio <= (res_w/res_h + 0.2):
            usable_files += [[file,tag]]

 47%|████▋     | 1025/2200 [00:09<00:11, 103.52it/s]/usr/local/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 2200/2200 [00:17<00:00, 122.96it/s]


In [46]:
im = Image.open(usable_files[0][0])

im_array = np.array(im)

In [47]:
im_array.shape

(768, 1024, 3)

In [48]:
idx = file.split('/')[1][:3]
idx

'087'

In [49]:
files_to_save_to_amazon = []
complete_files_to_save_to_amazon = []

for [file, tag] in tqdm(usable_files):
    im = Image.open(file)
    
    im_array = np.array(im)
    
    h = im_array.shape[0]
    v = im_array.shape[1]
    
    each_piece_h = int(h/4)
    each_piece_v = int(v/4)
    
    image_name = file.split('/')[-1]
    
    idx = file.split('/')[1][:3]
    
    if idx in pitman_list:
        path_to_save = 'chopped_images/' + tag + '/' + image_name[:-4]
        if not os.path.isdir(path_to_save):
            os.mkdir(path_to_save)

            im.save(path_to_save + '/' + image_name[:-4] + '_total.jpg')
            
            complete_files_to_save_to_amazon += [path_to_save + '/' + image_name[:-4] + '_total.jpg']

            count_of_crops = 0
            for i in range(4):
                for j in range(4):
                    cropped_im = im.crop((each_piece_v*i, each_piece_h*j, each_piece_v*(i+1), each_piece_h*(j+1)))
                    cropped_im.save(path_to_save+ '/' + image_name[:-4] 
                                    +'_{}.jpg'.format(count_of_crops))
                    
                    files_to_save_to_amazon += [path_to_save+ '/' + image_name[:-4] 
                                    +'_{}.jpg'.format(count_of_crops)]

                    count_of_crops += 1

 69%|██████▊   | 48/70 [00:09<00:04,  4.88it/s]/usr/local/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 70/70 [00:13<00:00,  5.10it/s]


In [31]:
file = usable_files[0][0]

In [50]:
for i in range(4):
    for j in range(4):
        cropped_im = im.crop((each_piece_h*i, each_piece_v*j, each_piece_h*(i+1), each_piece_v*(j+1)))
        cropped_im.save(path_to_save+ '/' + image_name[:-4] 
                                    +'_{}.jpg'.format(count_of_crops))
                    
        files_to_save_to_amazon += [path_to_save+ '/' + image_name[:-4] 
                                    +'_{}.jpg'.format(count_of_crops)]
        
        print(np.array(cropped_im).shape)

(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)
(200, 176, 3)


In [67]:
complete_files_list = [name.split('/')[1] + '/' + name.split('/')[3][:-10] for name in complete_files_to_save_to_amazon]

In [75]:
with open('martin_crazy_list.txt','w') as f:
    f.write('\n'.join(complete_files_list))

In [76]:
bucket_name = 'filament-hackathon-images'

# Uploads the given file using a managed uploader, which will split up large
# files automatically and upload parts in parallel.
s3.upload_file('martin_crazy_list.txt', bucket_name, 'martin_crazy_list.txt')

In [51]:
import boto3
import os

# Create an S3 client
s3 = boto3.client('s3')

for filename in complete_files_to_save_to_amazon:
    bucket_name = 'filament-hackathon-images'

    # Uploads the given file using a managed uploader, which will split up large
    # files automatically and upload parts in parallel.
    s3.upload_file(filename, bucket_name, filename)

In [52]:
for filename in tqdm(files_to_save_to_amazon):
    bucket_name = 'filament-hackathon-images'

    # Uploads the given file using a managed uploader, which will split up large
    # files automatically and upload parts in parallel.
    s3.upload_file(filename, bucket_name, filename)

 24%|██▍       | 276/1136 [00:09<00:30, 27.83it/s]/usr/local/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 1136/1136 [00:48<00:00, 23.55it/s]
